In [0]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [0]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [0]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

In [0]:
!unzip concrete_data_week3.zip

## Define Global Constants

Here, we will define constants that we will be using throughout the rest of the lab. 

1. We are obviously dealing with two classes, so *num_classes* is 2. 
2. The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3. We will training and validating the model using batches of 100 images.

In [0]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

## Construct ImageDataGenerator Instances

In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to *preprocess_input* which we imported from **keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Next, we will use the *flow_from_directory* method to get the training images as follows:

In [0]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [0]:
## Type your answer here
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 10001 images belonging to 2 classes.


## Build, Compile and Fit Model

In [0]:
model = Sequential()

Next, we will add the ResNet50 pre-trained model to out model. However, note that we don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument *include_top* and set it to **False**.

In [0]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

94658560/94653016 [==============================] - 4s 0us/step


Then, we will define our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.

In [0]:
model.add(Dense(num_classes, activation='softmax'))

You can access the model's layers using the *layers* attribute of our model object. 

In [0]:
model.layers

You can see that our model is composed of two sets of layers. The first set is the layers pertaining to ResNet50 and the second set is a single layer, which is our Dense layer that we defined above.

You can access the ResNet50 layers by running the following:

In [0]:
#model.layers[0].layers

Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense output layer. To do that, we run the following.

In [0]:
model.layers[0].trainable = False

And now using the *summary* attribute of the model, we can see how many parameters we will need to optimize in order to train the output layer.

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Next we compile our model using the **adam** optimizer.

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically, that is the number of images divided by the batch size. Therefore, we define our steps per epoch as follows:

In [0]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 10

Finally, we are ready to start training our model. Unlike a conventional deep learning training were data is not streamed from a directory, with an ImageDataGenerator where data is augmented in batches, we use the **fit_generator** method.

In [0]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/10
301/301 [==============================] - 147s 490ms/step - loss: 0.0362 - accuracy: 0.9893 - val_loss: 0.0246 - val_accuracy: 0.8797
Epoch 2/10
301/301 [==============================] - 136s 453ms/step - loss: 0.0151 - accuracy: 0.9961 - val_loss: 5.2452e-06 - val_accuracy: 0.9304
Epoch 3/10
301/301 [==============================] - 135s 449ms/step - loss: 0.0115 - accuracy: 0.9966 - val_loss: 2.1980e-04 - val_accuracy: 0.9303
Epoch 4/10
301/301 [==============================] - 136s 452ms/step - loss: 0.0078 - accuracy: 0.9980 - val_loss: 0.0130 - val_accuracy: 0.9390
Epoch 5/10
301/301 [==============================] - 136s 452ms/step - loss: 0.0086 - accuracy: 0.9979 - val_loss: 0.0000e+00 - val_accuracy: 0.9672
Epoch 6/10
301/301 [==============================] - 137s 454ms/step - loss: 0.0047 - accuracy: 0.9988 - val_loss: 6.7949e-06 - val_accuracy: 0.9653
Epoch 7/10
301/301 [==============================] - 136s 451ms/step - loss: 0.0054 - accuracy: 0.9983 - va

In [0]:
model.save('classifier_resnet50_model.h5')